# SCRIPT FOR CORRELATION OF YIELD TO PARAMETERS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import statsmodels.formula.api as sm
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
from sklearn.metrics import mean_squared_error
from math import sqrt
from functools import reduce
import numpy as np

%matplotlib inline

In [3]:
# Graphing parameters
#sns.set(style="white")
rc = {'figure.figsize':(5,5),
      'axes.facecolor':'white',
      'axes.grid' : False,
      'grid.color': '.8',
      'font.family':'Arial',
      'font.size' : 15,
      'figure.dpi':300}

plt.rcParams.update(rc)

## RAINFED CLIMATE PARAMETERS

In [15]:
evi = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_evi17.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'evi'})
pcp = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_pcp.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'pcp'})
lstday = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_LSTDAY.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'lstday'})
lstnight = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_LSTNIGHT.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'lstnight'})
et = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_ET.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'et'})
pet = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_PET.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'pet'})
slp = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_slope.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'slope'})
ele = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/ZonalSt_rf_ws_elevation.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'elevation'})
df_merged = [pcp, lstday, lstnight, et, pet, slp, ele, evi]
zonalStats = reduce(lambda  left,right: pd.merge(left,right,on=['ADM3_EN'],
                                            how='outer'), df_merged)
zonalStats.iloc[16,4]=zonalStats.iloc[44,4] # Duenas ET
zonalStats.iloc[16,0]='Dueñas' # Duenas label
zonalStats = zonalStats.drop(index=44)
#zonalStats.head()

dataOfficial = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed.csv')
yieldData = dataOfficial[(dataOfficial.param == 'yield') & (dataOfficial.ecosystem == 'rf')].reset_index(drop=True)
yieldData = yieldData[(yieldData.year <= 2017) & (yieldData.year >= 2001)].reset_index(drop=True).groupby('muni').mean()
#yieldData.head()

merged = pd.merge(yieldData, zonalStats, left_on='muni', right_on='ADM3_EN')
merged.dropna(inplace=True)
merged.head()
df=merged

# Alternative Variables
df['log_evi']=np.log(df['evi'])
df['exp_evi']=np.exp(df['evi'])
df['ratioday']=df['pcp']/df['lstday']
df['ratioday1']=df['lstday']/df['pcp']
df['rationight']=df['pcp']/df['lstnight']
df['rationight1']=df['lstnight']/df['pcp']
df['lstdiff']=df['lstday']-df['lstnight']
df['lstdiff1']=df['et']/(df['lstday']-df['lstnight'])
df['water_deficit']=df['et']-(df['pcp'])
df['water_deficit1']=np.log(df['et']-(df['pcp']))
#df['water_deficit2']=numpy.exp(df['pcp'])

# evi ~ pcp + lstday + lstnight + et + pet + elevation + slope
results = sm.ols(formula = "value ~  pcp + lstday + lstnight + et + pet + elevation", 
                 missing='drop',
                 data=df).fit()
print(results.summary())
print(results.params)

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     2.287
Date:                Thu, 09 Dec 2021   Prob (F-statistic):              0.152
Time:                        00:41:28   Log-Likelihood:                 5.1114
No. Observations:                  14   AIC:                             3.777
Df Residuals:                       7   BIC:                             8.251
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -13.0795     20.453     -0.639      0.5

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


## IRRIGATED CLIMATE PARAMETERS

In [5]:
evi = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_evi_17year.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'evi'})
pcp = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_pcp_17year.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'pcp'})
lstday = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_LSTDay_17year.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'lstday'})
lstnight = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_LSTNight_17year.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'lstnight'})
et = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_ET_17year.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'et'})
pet = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_PET_17year.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'pet'})
slp = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_ch3_slope.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'slope'})
ele = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/irr_ws_ch3_elevation.tif.csv', na_values='#NUM!', usecols=['ADM3_EN','MEAN']).rename(columns={'MEAN':'elevation'})
df_merged = [pcp, lstday, lstnight, et, pet, slp, ele, evi]
zonalStats = reduce(lambda  left,right: pd.merge(left,right,on=['ADM3_EN'],
                                            how='outer'), df_merged)
#zonalStats.iloc[16,4]=zonalStats.iloc[44,4] # Duenas ET
#zonalStats.iloc[16,0]='Dueñas' # Duenas label
#zonalStats = zonalStats.drop(index=44)
zonalStats.head()


dataOfficial = pd.read_csv('/Users/delatorredm/OneDrive - The University of Auckland/Ch4_Climate_Impacts/iloilo_rice_data_cleaned_imputed.csv')
yieldData = dataOfficial[(dataOfficial.param == 'yield') & (dataOfficial.ecosystem == 'irri')].reset_index(drop=True)
yieldData = yieldData[(yieldData.year <= 2017) & (yieldData.year >= 2001)].reset_index(drop=True).groupby('muni').mean()
yieldData.head()

merged = pd.merge(yieldData, zonalStats, left_on='muni', right_on='ADM3_EN')
merged.dropna(inplace=True)
merged.head()
df=merged

# Alternative Variables
df['log_evi']=np.log(df['evi'])
df['exp_evi']=np.exp(df['evi'])
df['ratioday']=df['pcp']/df['lstday']
df['ratioday1']=df['lstday']/df['pcp']
df['rationight']=df['pcp']/df['lstnight']
df['rationight1']=df['lstnight']/df['pcp']
df['lstdiff']=df['lstday']-df['lstnight']
df['lstdiff1']=df['et']/(df['lstday']-df['lstnight'])
df['water_deficit']=df['et']-(df['pcp'])
df['water_deficit1']=np.log(df['et']-(df['pcp']))
df['water_deficit2']=np.exp(df['pcp'])
df['squared']=df['pcp']*df['pcp']

# evi ~ pcp + lstday + lstnight + et + pet + elevation + slope
results = sm.ols(formula = "evi ~ pcp + lstnight + et + pet + slope", 
                 missing='drop',
                 data=df).fit()
print(results.summary())
#print(results.pvalues)

                            OLS Regression Results                            
Dep. Variable:                    evi   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.582
Method:                 Least Squares   F-statistic:                     2.669
Date:                Tue, 07 Dec 2021   Prob (F-statistic):              0.433
Time:                        18:35:44   Log-Likelihood:                 32.661
No. Observations:                   7   AIC:                            -53.32
Df Residuals:                       1   BIC:                            -53.65
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1028      0.857      3.619      0.1

/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/delatorredm/miniconda3/envs/ee/lib/python3.8/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


,year,value,ADM3_EN,pcp,lstday,lstnight,et,pet,slope,elevation,evi
0,2009,4.126261,Anilao,209.929624,29.431139,22.974285,38.524091,64.263255,0.737637,22.394737,0.434402
4,2009,4.301748,Dingle,212.941257,30.535767,22.864689,32.420334,51.524070,0.580816,21.881057,0.440071
7,2009,4.080896,Janiuay,202.895338,30.586455,22.419291,35.046648,54.355369,1.015070,46.500000,0.434068
10,2009,4.429136,Mina,207.031555,29.953373,22.731446,36.942835,58.185534,0.405093,32.341969,0.452257
11,2009,3.868858,New Lucena,202.664808,29.545483,22.699229,35.760819,55.777707,0.477912,20.000000,0.427145
15,2009,4.179030,San Miguel,197.762865,31.229186,22.821078,37.068107,63.982089,0.386685,31.040404,0.424204
16,2009,4.350423,Santa Barbara,196.363432,30.606100,22.670787,36.336826,63.233578,0.551387,24.544444,0.440007
